In [1]:
import warnings
import numpy as np
import pandas as pd
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
import csv
warnings.filterwarnings("ignore")

In [12]:
train_df = pd.read_csv('task2_trainset.csv', encoding = "utf8")
test_df = pd.read_csv('task2_public_testset.csv', encoding = "utf8")

In [111]:
train_df.head()

,Id,Title,Abstract,Authors,Categories,Created Date,Task 2,THEORETICAL,ENGINEERING,EMPIRICAL,OTHERS,Categories_embed_theoretical,Categories_embed_engineering,Categories_embed_empirical,Categories_embed_others
0,D00001,A Brain-Inspired Trust Management Model to Ass...,Rapid popularity of Internet of Things (IoT) a...,Mahmud/Kaiser/Rahman/Rahman/Shabut/Al-Mamun/Hu...,cs.CR/cs.AI/q-bio.NC,2018-01-11,THEORETICAL,1,0,0,0,0.194131,0.550785,0.484338,0.004624
1,D00002,On Efficient Computation of Shortest Dubins Pa...,"In this paper, we address the problem of compu...",Sadeghi/Smith,cs.SY/cs.RO/math.OC,2016-09-21,THEORETICAL,1,0,0,0,0.941462,0.525704,0.020966,0.000023
2,D00003,Data-driven Upsampling of Point Clouds,High quality upsampling of sparse 3D point clo...,Zhang/Jiang/Yang/Yamakawa/Shimada/Kara,cs.CV,2018-07-07,ENGINEERING,0,1,0,0,0.288054,0.660594,0.315537,0.025039
3,D00004,Accessibility or Usability of InteractSE? A He...,Internet is the main source of information now...,Aqle/Khowaja/Al-Thani,cs.HC,2018-08-29,EMPIRICAL,0,0,1,0,0.454069,0.519529,0.298405,0.057379
4,D00005,Spatio-Temporal Facial Expression Recognition ...,Automated Facial Expression Recognition (FER) ...,Hasani/Mahoor,cs.CV,2017-03-20,ENGINEERING,0,1,0,0,0.288054,0.660594,0.315537,0.025039


In [84]:
test_df.head()

,Id,Title,Abstract,Authors,Categories,Created Date,Categories_embed_theoretical
0,T00001,Cheating-Resilient Incentive Scheme for Mobile...,Mobile Crowdsensing is a promising paradigm fo...,Zhao/Yang/Yu/Yao/Lin/Li,cs.NI/cs.CR,2017-01-08,0.337862
1,T00002,10 Inventions on Key Guides and Keyboard Templ...,A keyboard has many function keys and each fun...,Mishra,cs.HC,2013-10-11,0.428744
2,T00003,Approximating Flexibility in Distributed Energ...,With increasing availability of communication ...,Kundu/Kalsi/Backhaus,cs.SY/math.OC,2018-03-14,0.901140
3,T00004,A New Perspective on Clustered Planarity as a ...,The clustered planarity problem (c-planarity) ...,Bläsius/Rutter,cs.DS/cs.DM/math.CO,2015-06-18,0.917090
4,T00005,Multitask Evolution with Cartesian Genetic Pro...,We introduce a genetic programming method for ...,Scott/De Jong,cs.NE,2017-02-07,0.355693


In [8]:
s = train_df["Task 2"].value_counts(normalize=True)
print(s)

THEORETICAL                          0.280714
ENGINEERING                          0.271429
EMPIRICAL                            0.144286
THEORETICAL ENGINEERING              0.105143
ENGINEERING EMPIRICAL                0.087571
THEORETICAL EMPIRICAL                0.053571
OTHERS                               0.037000
THEORETICAL ENGINEERING EMPIRICAL    0.020286
Name: Task 2, dtype: float64


In [45]:
train_df["THEORETICAL"] = train_df["Task 2"].where(train_df["Task 2"].str.contains("THEORETICAL"), 0)
train_df["ENGINEERING"] = train_df["Task 2"].where(train_df["Task 2"].str.contains("ENGINEERING"), 0)
train_df["EMPIRICAL"] = train_df["Task 2"].where(train_df["Task 2"].str.contains("EMPIRICAL"), 0)
train_df["OTHERS"] = train_df["Task 2"].where(train_df["Task 2"].str.contains("OTHERS"), 0)

In [46]:
train_df["THEORETICAL"].loc[train_df["THEORETICAL"] != 0] = 1
train_df["ENGINEERING"].loc[train_df["ENGINEERING"] != 0] = 1
train_df["EMPIRICAL"].loc[train_df["EMPIRICAL"] != 0] = 1
train_df["OTHERS"].loc[train_df["OTHERS"] != 0] = 1

In [89]:
df = train_df.append(test_df)

# embed "Categories"

In [67]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [91]:
vocab_size = 200
encoded_docs = [one_hot(d, vocab_size) for d in df["Categories"]]
max_length = 20
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
train_padded_docs = padded_docs[:7000]
test_padded_docs = padded_docs[7000:]

In [75]:
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

Instructions for updating:
Colocations handled automatically by placer.


   ## embedded to THEORETICAL

In [101]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_padded_docs, train_df["THEORETICAL"], epochs=100, verbose=0)
loss, accuracy = model.evaluate(train_padded_docs, train_df["THEORETICAL"], verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 72.971429


In [102]:
test_df["Categories_embed_theoretical"] = model.predict(test_padded_docs)
train_df["Categories_embed_theoretical"] = model.predict(train_padded_docs)

# embedded to ENGINEERING

In [103]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_padded_docs, train_df["ENGINEERING"], epochs=100, verbose=0)
loss, accuracy = model.evaluate(train_padded_docs, train_df["ENGINEERING"], verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 66.171429


In [104]:
test_df["Categories_embed_engineering"] = model.predict(test_padded_docs)
train_df["Categories_embed_engineering"] = model.predict(train_padded_docs)

## embedded to EMPIRICAL

In [105]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_padded_docs, train_df["EMPIRICAL"], epochs=100, verbose=0)
loss, accuracy = model.evaluate(train_padded_docs, train_df["EMPIRICAL"], verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 74.500000


In [106]:
test_df["Categories_embed_empirical"] = model.predict(test_padded_docs)
train_df["Categories_embed_empirical"] = model.predict(train_padded_docs)

## embedded to OTHERS

In [108]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(train_padded_docs, train_df["OTHERS"], epochs=100, verbose=0)
loss, accuracy = model.evaluate(train_padded_docs, train_df["OTHERS"], verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 96.728571


In [109]:
test_df["Categories_embed_others"] = model.predict(test_padded_docs)
train_df["Categories_embed_others"] = model.predict(train_padded_docs)

# simple training with embedded data

In [117]:
trainning = train_df[["Categories_embed_theoretical","Categories_embed_engineering","Categories_embed_empirical","Categories_embed_others"]]
testing = test_df[["Categories_embed_theoretical","Categories_embed_engineering","Categories_embed_empirical","Categories_embed_others"]]


In [110]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.externals.six import StringIO   
from sklearn.tree import export_graphviz

import pydotplus

In [141]:
dtree=DecisionTreeClassifier(max_depth=3)
dtree.fit(trainning, train_df["OTHERS"])

dot_data = StringIO()
export_graphviz(dtree, 
                out_file=dot_data,  
                filled=True, 
                feature_names=list(trainning),
                class_names=['no','yes'],
                special_characters=True)

graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
graph.write_pdf("tree.pdf")

True

In [135]:
output = pd.DataFrame()
output = pd.read_csv('task2_sample_submission.csv', encoding = "utf8")

In [136]:
output["THEORETICAL"][:20000] = dtree.predict(testing)

In [138]:
output["ENGINEERING"][:20000] = dtree.predict(testing)

In [140]:
output["EMPIRICAL"][:20000] = dtree.predict(testing)

In [142]:
output["OTHERS"][:20000] = dtree.predict(testing)

In [144]:
output.head()

,order_id,THEORETICAL,ENGINEERING,EMPIRICAL,OTHERS
0,T00001,0,1,0,0
1,T00002,0,0,0,0
2,T00003,1,0,0,0
3,T00004,1,0,0,0
4,T00005,0,1,0,0


In [145]:
output.to_csv("submit.csv", index = False)